In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
import pandas as pd

# Load the Excel file
excel_file = 'Prev_results/lexicons_firefox.xlsx'  # Replace with your Excel file path

# Read the two sheets
df_severe = pd.read_excel(excel_file, sheet_name='Severe Lexicons')  # Replace with your sheet name if different
df_nonsevere = pd.read_excel(excel_file, sheet_name='NonSevere Lexicons')  # Replace with your sheet name if different

# Drop the 'Ratio' column if it exists
if 'Ratio' in df_severe.columns:
    df_severe = df_severe.drop(columns=['Ratio'])
if 'Ratio' in df_nonsevere.columns:
    df_nonsevere = df_nonsevere.drop(columns=['Ratio'])

# Convert the DataFrames to text files
df_severe.to_csv('severe_threshold_F.txt', sep='\t', index=False)
df_nonsevere.to_csv('nonsevere_threshold_F.txt', sep='\t', index=False)

print("Excel sheets have been converted to 'severe_lsvm.txt' and 'nonsevere_lsvm.txt' without the 'Ratio' column")


Excel sheets have been converted to 'severe_lsvm.txt' and 'nonsevere_lsvm.txt' without the 'Ratio' column


In [3]:
# # Load positive and negative words from Bing Liu Lexicon
# with open('positive-words.txt', 'r') as f:
#     words = set(f.read().splitlines())

# with open('negative-words.txt', 'r') as f:
#     negative_words = set(f.read().splitlines())

In [4]:
bugs_eclipse = pd.read_csv("bugs_eclipse.csv")


bugs_eclipse['Type'] = np.where(bugs_eclipse['Severity'] == 'enhancement', "enhancement", "defect")


# Dropped rows with severity level '--'
bugs_eclipse = bugs_eclipse[bugs_eclipse["Severity"].str.contains("--")==False].reset_index()

#Dropped rows with Type "Enhancement" and "Task" because they are not a bug but a new feature
indexSevere = bugs_eclipse[(bugs_eclipse['Type'] == 'enhancement') & (bugs_eclipse['Type'] == 'enhancement') ].index
bugs_eclipse.drop(indexSevere , inplace=True)

indexSevere = bugs_eclipse[(bugs_eclipse['Type'] == 'task') & (bugs_eclipse['Type'] == 'task') ].index
bugs_eclipse.drop(indexSevere , inplace=True)


#Catagorise the severity level into a Severe and Non Severe to make it a binary problem
bugs_eclipse.loc[bugs_eclipse["Severity"] == "blocker", "Severity"] = 'Severe'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "critical", "Severity"] = 'Severe'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "major", "Severity"] = 'Severe'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "S1", "Severity"] = 'Severe'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "S2", "Severity"] = 'Severe'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "S3", "Severity"] = 'NonSevere'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "normal", "Severity"] = 'NonSevere'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "minor", "Severity"] = 'NonSevere'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "trivial", "Severity"] = 'NonSevere'
bugs_eclipse.loc[bugs_eclipse["Severity"] == "S4", "Severity"] = 'NonSevere'

df = bugs_eclipse

In [5]:
def check_word_intersection(txt_file, df, column_name, severity_column):
    # Read the words from the text file
    with open(txt_file, 'r') as file:
        words = file.read().splitlines()
    
    # Initialize a Counter to keep track of word intersections
    word_counter = Counter()
    
    # Dictionaries to keep track of words in severe and non-severe categories
    severe_words = Counter()
    non_severe_words = Counter()
    
    # Function to find intersecting words and update the counters
    def find_intersections(text, severity):
        text_words = set(text.split())
        intersections = text_words.intersection(words)
        for word in intersections:
            word_counter[word] += 1
            if severity == 'Severe':
                severe_words[word] += 1
            elif severity == 'NonSevere':
                non_severe_words[word] += 1
        return ', '.join(intersections) if intersections else None
    
    # Check for intersections in the DataFrame column
    df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
    # Words that appear in both categories
    both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
    # Words that appear only in severe category
    only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
    # Words that appear only in non-severe category
    only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
    # Return the DataFrame with the intersections column and the word counters
    return df, word_counter, both_categories, only_severe, only_non_severe

# Example usage:

# Assuming df is already defined and has columns named 'Summary' and 'Severity'
result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('severe_threshold_F.txt', df, 'Summary', 'Severity')
filtered_df = result_df[result_df['intersections'].notna()]

# Print the filtered DataFrame
# print(filtered_df['intersections'])
# print(set(filtered_df['intersections']))

# Print the word counts
print(word_counts)

# Print the categorized words
# print("Words in both categories:", both_categories)
print("Words only in severe category:", only_severe)
print("Words only in non-severe category:", only_non_severe)


Counter({'not': 4470, 'file': 1178, 'error': 1001, 'server': 870, 'wizard': 761, 'new': 581, 'web': 564, 'work': 494, 'missing': 477, 'fails': 461, 'build': 426, 'version': 410, 'using': 385, 'update': 361, 'service': 344, 'support': 329, 'cannot': 281, 'create': 276, 'message': 269, 'add': 268, 'extension': 264, 'model': 260, 'plugin': 257, 'open': 249, 'creating': 248, 'change': 245, 'show': 243, 'creation': 233, 'working': 223, 'properly': 221, 'problem': 216, 'workspace': 208, 'invalid': 202, 'import': 200, 'added': 196, 'empty': 192, 'adding': 190, 'created': 189, 'property': 187, 'broken': 182, 'opening': 177, 'editing': 173, 'value': 167, 'documentation': 160, 'performance': 159, 'found': 156, 'package': 152, 'generated': 146, 'publish': 144, 'run': 144, 'point': 141, 'function': 140, 'running': 135, 'start': 134, 'due': 133, 'available': 133, 'directory': 130, 'instead': 123, 'root': 122, 'node': 122, 'configuration': 122, 'one': 122, 'time': 121, 'existing': 119, 'first': 114,

In [6]:
def check_word_intersection(txt_file, df, column_name, severity_column):
    # Read the words from the text file
    with open(txt_file, 'r') as file:
        words = file.read().splitlines()
    
    # Initialize a Counter to keep track of word intersections
    word_counter = Counter()
    
    # Dictionaries to keep track of words in severe and non-severe categories
    severe_words = Counter()
    non_severe_words = Counter()
    
    # Function to find intersecting words and update the counters
    def find_intersections(text, severity):
        text_words = set(text.split())
        intersections = text_words.intersection(words)
        for word in intersections:
            word_counter[word] += 1
            if severity == 'Severe':
                severe_words[word] += 1
            elif severity == 'NonSevere':
                non_severe_words[word] += 1
        return ', '.join(intersections) if intersections else None
    
    # Check for intersections in the DataFrame column
    df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
    # Words that appear in both categories
    both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
    # Words that appear only in severe category
    only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
    # Words that appear only in non-severe category
    only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
    # Return the DataFrame with the intersections column and the word counters
    return df, word_counter, both_categories, only_severe, only_non_severe

# Example usage:

# Assuming df is already defined and has columns named 'Summary' and 'Severity'
result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('nonsevere_threshold_F.txt', df, 'Summary', 'Severity')
filtered_df = result_df[result_df['intersections'].notna()]

# Print the filtered DataFrame
# print(filtered_df)
# print(set(filtered_df['intersections']))

# Print the word counts
print(word_counts)

# Print the categorized words
# print("Words in both categories:", both_categories)
print("Words only in severe category:", only_severe)
print("Words only in non-severe category:", only_non_severe)


Counter({'not': 4470, 'project': 1627, 'file': 1178, 'editor': 795, 'validation': 770, 'page': 725, 'new': 581, 'view': 572, 'web': 564, 'work': 494, 'missing': 477, 'name': 456, 'type': 454, 'source': 438, 'content': 424, 'version': 410, 'class': 388, 'default': 387, 'using': 385, 'attribute': 385, 'update': 361, 'runtime': 355, 'service': 344, 'use': 344, 'exception': 338, 'code': 337, 'module': 329, 'support': 329, 'tag': 294, 'need': 287, 'cannot': 281, 'dialog': 279, 'incorrect': 274, 'test': 274, 'add': 268, 'extension': 264, 'schema': 259, 'plugin': 257, 'set': 251, 'open': 249, 'wrong': 248, 'change': 245, 'folder': 244, 'show': 243, 'preference': 237, 'client': 232, 'element': 226, 'validator': 225, 'working': 223, 'properly': 221, 'java': 217, 'correctly': 214, 'jar': 212, 'context': 209, 'text': 208, 'menu': 207, 'method': 203, 'do': 200, 'selection': 199, 'xml': 198, 'remove': 198, 'added': 196, 'plugins': 194, 'empty': 192, 'mapping': 190, 'created': 189, 'property': 187, 

In [ ]:
# import pandas as pd
# from collections import Counter

# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Debug prints to check the counts
#     print("Severe words count:", severe_words)
#     print("Non-severe words count:", non_severe_words)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('positive-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [ ]:
# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('positive-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [ ]:
# def check_word_intersection(txt_file, df, column_name, severity_column):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Dictionaries to keep track of words in severe and non-severe categories
#     severe_words = Counter()
#     non_severe_words = Counter()
    
#     # Function to find intersecting words and update the counters
#     def find_intersections(text, severity):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#             if severity == 'Severe':
#                 severe_words[word] += 1
#             elif severity == 'Non-Severe':
#                 non_severe_words[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df.apply(lambda row: find_intersections(row[column_name], row[severity_column]), axis=1)
    
#     # Words that appear in both categories
#     both_categories = {word for word in words if severe_words[word] > 0 and non_severe_words[word] > 0}
    
#     # Words that appear only in severe category
#     only_severe = {word for word in words if severe_words[word] > 0 and non_severe_words[word] == 0}
    
#     # Words that appear only in non-severe category
#     only_non_severe = {word for word in words if non_severe_words[word] > 0 and severe_words[word] == 0}
    
#     # Return the DataFrame with the intersections column and the word counters
#     return df, word_counter, both_categories, only_severe, only_non_severe

# # Example usage:

# # Assuming df is already defined and has columns named 'Summary' and 'Severity'
# result_df, word_counts, both_categories, only_severe, only_non_severe = check_word_intersection('negative-words.txt', df, 'Summary', 'Severity')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

# # Print the categorized words
# print("Words in both categories:", both_categories)
# print("Words only in severe category:", only_severe)
# print("Words only in non-severe category:", only_non_severe)


In [ ]:
# import pandas as pd
# from collections import Counter

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Initialize a Counter to keep track of word intersections
#     word_counter = Counter()
    
#     # Function to find intersecting words and update the counter
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         for word in intersections:
#             word_counter[word] += 1
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column and the word counter
#     return df, word_counter

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df, word_counts = check_word_intersection('positive-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(filtered_df)

# # Print the word counts
# print(word_counts)

In [ ]:
# bugs_df['Processed_Summary'] = bugs_df['Summary'].apply(lambda x: helper.nlpsteps(x))

In [ ]:

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Function to find intersecting words
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column
#     return df

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df = check_word_intersection('positive-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(set(filtered_df['intersections']))
# # print(result_df)


In [ ]:
# import pandas as pd

# def check_word_intersection(txt_file, df, column_name):
#     # Read the words from the text file
#     with open(txt_file, 'r') as file:
#         words = file.read().splitlines()
    
#     # Function to find intersecting words
#     def find_intersections(text):
#         text_words = set(text.split())
#         intersections = text_words.intersection(words)
#         return ', '.join(intersections) if intersections else None
    
#     # Check for intersections in the DataFrame column
#     df['intersections'] = df[column_name].apply(find_intersections)
    
#     # Return the DataFrame with the intersections column
#     return df

# # Example usage:

# # Assuming df is already defined and has a column named 'Summary'
# result_df = check_word_intersection('negative-words.txt', df, 'Summary')
# filtered_df = result_df[result_df['intersections'].notna()]

# # Print the filtered DataFrame
# print(set(filtered_df['intersections']))
# # print(result_df)
